<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTuning_LLM_Meta_Llama_3_8B_for_MEDAL_EVALDATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://llama.meta.com/docs/how-to-guides/fine-tuning/


https://medium.com/@frankmorales_91352/fine-tuning-meta-llama-3-8b-with-medal-a-refined-approach-for-enhanced-medical-language-b924d226b09d

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

! pip install trl==0.8.6 -q


In [2]:
! pip install trl==0.8.6 -q

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [4]:
from huggingface_hub import login

login(
  token=access_token_write, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from trl import SFTTrainer

In [3]:
# set device
device = 'cuda'

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

In [5]:
torch.__version__

'2.3.0+cu121'

In [ ]:
!apt-get update && apt-get install -y cuda-11-0

In [7]:
!python --version
!nvcc --version
!nvidia-smi

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Sun Jun 23 03:49:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0       

In [8]:
### conversational format
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}

### instruction format
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}

{'prompt': '<prompt text>', 'completion': '<ideal generated text>'}

In [9]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/train_dataset.json", split="train")

In [10]:
dataset

Dataset({
    features: ['abstract_id', 'text', 'location', 'label'],
    num_rows: 3000000
})

In [11]:
print(dataset[6]['text'])

to quantify the amount of CL NO harvested in modified RND


In [12]:
print(dataset[6]['label'])

['radical neck dissection']


In [13]:
dataset_test = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

In [14]:
dataset_test

Dataset({
    features: ['abstract_id', 'text', 'location', 'label'],
    num_rows: 1000000
})

In [15]:
import re

def dataset_to_finetuning(dataset,nrec):

    question=dataset['text']
    answer_entry=dataset['label']


    !rm -rf /content/question_answering.txt
    filename='/content/question_answering.txt'

    #nrec=8000


    # 423194 training
    # 10807 eval

  # python object to be appended
    for n in range(nrec):
        #print(answer[n])
        if answer_entry[n] == None:
           answer_entry[n] = 'Not possible to get or use'
        string = answer_entry[n]

        str_question=question[n]
        question_final= re.sub("\n", "", str_question)

        answer = re.sub("\[", "", str(string))
        answer = re.sub("\]", "", str(answer))
        answer0=answer[1:len(answer)-1]

        if len(answer0)==0:
            answer='Not possible to get or use'

        #print(answer0)


        text='<s>[INST] %s [/INST] %s </s>'%(question_final,answer0)
        del string,answer0
        #print(text)
        with open(filename, 'a') as f:
            f.write(text + '\n')
        f.close()

    text0 = load_dataset("text", data_files="/content/question_answering.txt", split="train")

    dataset_final_Question=dataset['text'][0:nrec]
    dataset_final_Answer=dataset['label'][0:nrec]
    dataset_final_text=text0[0:nrec]['text']

    import pandas as pd

    datasetF = pd.DataFrame() # Create an empty DataFrame
    datasetF['Question'] = dataset_final_Question
    datasetF['Answer'] = dataset_final_Answer
    datasetF['text'] = dataset_final_text


    return datasetF

In [16]:
#del datasetF
#datasetF=dataset_to_finetuning(dataset,423194)
datasetF=dataset_to_finetuning(dataset,500000)

Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
datasetF['Question'][6]

'to quantify the amount of CL NO harvested in modified RND'

In [18]:
datasetF['Answer'][6]

['radical neck dissection']

In [19]:
datasetF['text'][6]

'<s>[INST] to quantify the amount of CL NO harvested in modified RND [/INST] radical neck dissection </s>'

In [20]:
datasetF

,Question,Answer,text
0,velvet antlers vas are commonly used in tradit...,[transverse aortic constriction],<s>[INST] velvet antlers vas are commonly used...
1,the clinical features of our cases demonstrate...,[hodgkins lymphoma],<s>[INST] the clinical features of our cases d...
2,ceftobiprole bpr is an investigational cephalo...,[methicillinsusceptible s aureus],<s>[INST] ceftobiprole bpr is an investigation...
3,we have taken a basic biologic RPA to elucidat...,[parathyroid hormonerelated protein],<s>[INST] we have taken a basic biologic RPA t...
4,lipoperoxidationderived aldehydes for example ...,[lipoperoxidation],<s>[INST] lipoperoxidationderived aldehydes fo...
...,...,...,...
499995,whim warts hypogammaglobulinemia infections an...,[chemotaxis],<s>[INST] whim warts hypogammaglobulinemia inf...
499996,lipids either as membrane components or as fue...,[fractional outflow rate],<s>[INST] lipids either as membrane components...
499997,we studied the EP and antiarrhythmic effects o...,[attachment site],<s>[INST] we studied the EP and antiarrhythmic...
499998,the role of HR rate in the OD of coronary athe...,[high cholesterol],<s>[INST] the role of HR rate in the OD of cor...


In [21]:
#datasetF_test=dataset_to_finetuning(dataset_test,10807)
datasetF_test=dataset_to_finetuning(dataset_test,20000)

Generating train split: 0 examples [00:00, ? examples/s]

In [22]:
datasetF_test

,Question,Answer,text
0,we developed an animal model of chronic allerg...,[functional residual capacity],<s>[INST] we developed an animal model of chro...
1,pyogenic granulomas represent the aquisition o...,[pyogenic granuloma],<s>[INST] pyogenic granulomas represent the aq...
2,the l immunotype lipopolysaccharide lps of nei...,[phosphorylethanolamine],<s>[INST] the l immunotype lipopolysaccharide ...
3,inotropic reserve identified by dobutamine or ...,[stress echocardiography],<s>[INST] inotropic reserve identified by dobu...
4,pyridoxinedependent seizure is a rare autosoma...,[severe mental retardation],<s>[INST] pyridoxinedependent seizure is a rar...
...,...,...,...
19995,penile erection is regulated by two opposing s...,[corpus cavernosum],<s>[INST] penile erection is regulated by two ...
19996,effluent from oil production activities contai...,[hydrocarbons],<s>[INST] effluent from oil production activit...
19997,a new method is described for performing oral ...,[acute respiratory failure],<s>[INST] a new method is described for perfor...
19998,since more than years risk assessment of bisph...,[endocrine disruptors],<s>[INST] since more than years risk assessmen...


In [23]:
datasetF_test['Question'][6]

'while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in summar

In [24]:
datasetF_test['Answer'][6]

['antral follicle count']

In [25]:
datasetF_test['text'][6]

'<s>[INST] while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
#model_id = "mistralai/Mistral-7B-Instruct-v0.1" #01 march 2024 AND 10/03/2024

model_id = "meta-llama/Meta-Llama-3-8B" #22 MAY 2024

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id


In [27]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): 

In [28]:
from peft import LoraConfig

# LoRA config based on gemini session 22/06/2024
peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.05,
        r=128,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

A warm-up is in general an increase of the learning rate. It starts at 0 and then increases linearly over 1(here) step to the specified learning rate of 1.25e-5.

Afterwards by default a linear (in other cases a cosine) learning-rate scheduler decays your learning-rate.

To disable the decay add lr_scheduler_type='constant'. Iirc, this also disables the warmup. If you want warmup and afterwards a constant rate use constant_with_warmup instead.

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(

    output_dir="/content/gdrive/MyDrive/model/POC-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata",

    #num_train_epochs=3,                     # number of training epochs
    num_train_epochs=0.3,                    # number of training epochs for POC
    per_device_train_batch_size=1,          # batch size per device during training
    #2
    gradient_accumulation_steps=8,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    #gradient_checkpointing_kwargs={"use_reentrant": True},
    optim="adamw_torch_fused",              # use fused adamw optimizer

    #ELECTRA is trained with Adam optimizer with learning
    #rate of 0.00002 and with batch size of 16


    optim="adamw_torch_fused", learning_rate=5e-5, warmup_ratio=0.15, weight_decay=0.05,
    #trainer = Trainer(model=model, args=training_args, train_dataset=ds, optimizers=(adam_bnb_optim, None))
    logging_steps=100,                       # log every 10 steps
    #save_strategy="epoch",                  # save checkpoint every epoch

    learning_rate=5e-5,                     # learning rate, based on QLoRA paper # i used in the first model
    #learning_rate=1.41e-6,
    # --lr 2e-6

    #Common Range: The most common range for fine-tuning learning rates is between 1e-5 and 1e-3.
    #However, the optimal value will depend on various factors, including the model architecture,
    #the dataset size, and the specific fine-tuning task.

    #learning_rate=1.41e-5,
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=1.0,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.15,                      # warmup ratio based on QLoRA paper = 0.03

    weight_decay=0.05,
    #Here's why 0.03 is a good initial choice:
#Common Practice: It's a value that's frequently used and has been shown to work well in a variety of situations.
#Balance: It offers a good balance between giving the model time to warm up and not spending too much of your training budget on the warmup phase.
#Flexibility: You can always adjust the warmup_ratio up or down based on the specific characteristics of your model and dataset.

    #Yes, a `warmup_ratio` of 0.03 is generally a good starting point for many training scenarios.
    #It's a common and well-established practice that can help stabilize training and improve the performance of your model.

    #lr_scheduler_type="constant",           # use constant learning rate scheduler
    lr_scheduler_type="cosine",            # lr_scheduler_type="cosine" (Cosine Annealing Learning Rate)

    #In many cases, cosine annealing often outperforms a constant learning rate due
    #to its ability to adapt the learning rate during training.
    #This can lead to faster convergence and better model performance.

    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    gradient_checkpointing_kwargs={"use_reentrant": True},

    load_best_model_at_end=True,
    ## NEW SINCE 21/06/2024
    #evaluation_strategy="epoch",
    logging_dir="/content/gdrive/MyDrive/model/POC-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata/logs",

    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    metric_for_best_model = "loss",
    #metric_for_best_model="exact_match",
)

In [30]:
!pip install trl -q

In [31]:
model.config.use_cache=False
model.gradient_checkpointing_enable() #enable gradient checkpoint
#torch.utils.checkpoint.checkpoint(use_reentrant=False)

https://huggingface.co/docs/trl/main/en/sft_trainer

In [ ]:
from trl import SFTTrainer
import datasets
#from datasets import load_metric

# Convert Pandas DataFrame to Hugging Face Dataset
datasetF_hf = datasets.Dataset.from_pandas(datasetF)
datasetF_hf_test = datasets.Dataset.from_pandas(datasetF_test)

#max_seq_length = 3072 # max sequence length for model and packing of the dataset
max_seq_length = 1024

# Explicitly set the device using torch.device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
#model.to(device)

# Add a padding token to your tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Or, add a new padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

#metric = load_metric("squad")  # Assuming you're evaluating on SQuAD dataset

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)



trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=datasetF_hf,
    dataset_text_field="text", ### added for the summarization dataset
    eval_dataset=datasetF_hf_test,

    #compute_metrics=compute_metrics,
    #Custom Metrics: You would only need to define a compute_metrics function
    #if you wanted to track and use custom metrics for evaluation, such as perplexity,
    #BLEU score, or task-specific metrics.

    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,

    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
    }
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py

In [33]:
#/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download`
#is deprecated and will be removed in version 1.0.0. Downloads always resume when possible.
#If you want to force a new download, use `force_download=True`.

#!pip show huggingface_hub

#Upgrade: If you have an older version (anything below 1.0.0), upgrade to the latest version using:

#!pip install --upgrade huggingface_hub

In [ ]:
from transformers import EarlyStoppingCallback


trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=5))


# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model()

# [3668/7230 13:32:23 < 13:09:20, 0.08 it/s, Epoch 0.51/1]

# Step	Training Loss	Validation Loss
# 200	  2.505300	    2.382469
# 3600	2.226800	    2.223289

In [35]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

Test Model and run Inference

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

peft_model_id = "/content/gdrive/MyDrive/model/POC-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata"
#peft_model_id = 'frankmorales2020/NEW-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata'


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  attn_implementation="flash_attention_2",
  quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Let’s load our test dataset try to generate an instruction.

In [37]:
print(model.peft_config)

{'default': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='meta-llama/Meta-Llama-3-8B', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=128, target_modules={'gate_proj', 'v_proj', 'q_proj', 'down_proj', 'o_proj', 'up_proj', 'k_proj'}, lora_alpha=64, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)}


In [ ]:
from datasets import load_dataset
from random import randint


# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")
nrec= randint(0, len(eval_dataset))
nrec=6

# Test on sample
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map
prompt =  eval_dataset[nrec]['text']


outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [39]:
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
ganswer

'antral follicle count </s><s>[INST] the aim of this study was to evaluate the effect of the addition of a new antifungal agent to the standard antifungal regimen in the treatment of invasive fungal infections ifi in patients with acute leukemia al and HSCT hsct recipients [/INST] hematopoietic stem cell transplant </s><s>[INST] the aim of this study was to evaluate the effect of the addition of a new antifungal agent to the standard antifungal regimen in the treatment of invasive fungal infections ifi in patients with acute leukemia al and HSCT hsct'

In [40]:
print(f"Query:\n{eval_dataset[nrec]['text']}")
print()
oanswer=str(eval_dataset[nrec]['label'])
oanswer=oanswer[2:len(oanswer)-2]
print(f"Original Answer:\n{oanswer}")
print()
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
qc=str(ganswer).find('[INST]')
ganswer=ganswer[0:qc-7]
qc0=str(ganswer).find('[INST]')
ganswer=str(ganswer)[0:qc0]
qc=str(ganswer).find('[/INST]')
if qc>0:
  ganswer=ganswer[qc+8:len(ganswer)]
print(f"Generated Answer:\n{ganswer}")
print()
if ganswer == oanswer:
  print("Match")
else:
  print("NO Match")

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [41]:
nrec=6
print(f"Query:\n{eval_dataset[nrec]['text']}")
print()
print(f"Original Answer:\n{eval_dataset[nrec]['label']}")
print()
print(f"Full Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [42]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc=str(ganswer).find('[/INST]')
#print(qc)
if int(qc)<0:
    #print(f"Generated Answer:\n{ganswer}")
    ganswer=outputs[0]['generated_text'][len(prompt)+8:].strip()
else:
    ganswer=ganswer[qc:len(ganswer)]
ganswer

'[/INST] antral follicle count </s><s>[INST] the aim of this study was to evaluate the effect of the addition of a new antifungal agent to the standard antifungal regimen in the treatment of invasive fungal infections ifi in patients with acute leukemia al and HSCT hsct recipients [/INST] hematopoietic stem cell transplant </s><s>[INST] the aim of this study was to evaluate the effect of the addition of a new antifungal agent to the standard antifungal regimen in the treatment of invasive fungal infections ifi in patients with acute leukemia al and HSCT hsct'

In [43]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc0=str(ganswer).find('[INST]')
ganswer=str(ganswer)[0:qc0-8]
qc=str(ganswer).find('[/INST]')

if qc>=0:
  ganswer=ganswer[qc+8:len(ganswer)]

print(f"Generated Answer:\n{ganswer}")

Generated Answer:
antral follicle count
